In [28]:
import pandas as pd
import numpy as np
import sqlite3 as sql

In [42]:
database = 'SB_DB.db'
connection = sql.connect(database)

### TABLES AND COLUMNS

**Bookings:** bookid, facid, memid, starttime, slots<br>
**Facilities:** facid, name, membercost, guestcost, initialoutlay, monthlymaintenance<br>
**Members:** memid, surname, firstname, address, zipcode, telephone, recommendedby, joindate

### QUESTIONS

#### Q1: Some of the facilities charge a fee to members, but some do not. Write a SQL query to produce a list of the names of the facilities that do.

In [323]:
query = '''
SELECT name
FROM Facilities
WHERE membercost != 0'''

In [324]:
df = pd.read_sql_query(query, connection)
df

,name
0,Tennis Court 1
1,Tennis Court 2
2,Massage Room 1
3,Massage Room 2
4,Squash Court


<br>

#### Q2: How many facilities do not charge a fee to members?

In [325]:
query = '''
SELECT COUNT(name) as Total
FROM Facilities
WHERE membercost = 0'''

In [326]:
df = pd.read_sql_query(query, connection)
df

,Total
0,4


<br>

#### Q3: Write an SQL query to show a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost. Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [327]:
query = """
SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE membercost != 0 AND membercost/monthlymaintenance < 0.2 """

In [329]:
df = pd.read_sql_query(query, connection)
df

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,4,Massage Room 1,9.9,3000
3,5,Massage Room 2,9.9,3000
4,6,Squash Court,3.5,80


<br>

#### Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5. Try writing the query without using the OR operator.

In [330]:
query = """
SELECT *
FROM Facilities
WHERE facid IN (1,5)"""

In [331]:
df = pd.read_sql_query(query, connection)
df

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


<br>

#### Q5: Produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100. Return the name and monthly maintenance of the facilities in question.

In [341]:
query = """
SELECT name, monthlymaintenance,
      CASE WHEN monthlymaintenance > 100 THEN 'expensive'
           ELSE 'cheap' END AS label
FROM Facilities
"""

In [342]:
df = pd.read_sql_query(query, connection)
df

,name,monthlymaintenance,label
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


<br>

#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Try not to use the LIMIT clause for your solution.

In [334]:
query = """SELECT firstname, surname
FROM Members
WHERE joindate = (SELECT MAX(joindate)
                  FROM Members)"""

In [335]:
df = pd.read_sql_query(query, connection)
df

,firstname,surname
0,Darren,Smith


<br>

#### Q7: Produce a list of all members who have used a tennis court. Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name.

In [345]:
query = """
SELECT f.name, (m.surname || ', ' || m.firstname) AS member_name
FROM Bookings AS b
LEFT JOIN Members AS m
ON m.memid = b.memid
LEFT JOIN Facilities AS f
ON f.facid = b.facid
WHERE f.name LIKE ('Tennis%')
GROUP BY f.name, member_name
ORDER BY member_name"""


In [347]:
df = pd.read_sql_query(query, connection)
df.head()

,name,member_name
0,Tennis Court 1,"Bader, Florence"
1,Tennis Court 2,"Bader, Florence"
2,Tennis Court 1,"Baker, Anne"
3,Tennis Court 2,"Baker, Anne"
4,Tennis Court 1,"Baker, Timothy"


<br>

#### Q8: Produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30. Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.

In [359]:
query = """
SELECT f.name AS booking, (m.surname || ', ' || m.firstname) AS member_name,
        CASE WHEN m.memid = 0 THEN guestcost*slots
            ELSE membercost*slots END AS cost_facility
FROM Bookings AS b
LEFT JOIN Members AS m
ON m.memid = b.memid
LEFT JOIN Facilities AS f
ON f.facid = b.facid
WHERE starttime BETWEEN '2012-09-14 00:00:00' AND '2012-09-14 23:59:59' AND cost_facility > 30
ORDER BY cost_facility DESC"""

In [360]:
df = pd.read_sql_query(query, connection)
df

,booking,member_name,cost_facility
0,Massage Room 2,"GUEST, GUEST",320.0
1,Massage Room 1,"GUEST, GUEST",160.0
2,Massage Room 1,"GUEST, GUEST",160.0
3,Massage Room 1,"GUEST, GUEST",160.0
4,Tennis Court 2,"GUEST, GUEST",150.0
5,Tennis Court 1,"GUEST, GUEST",75.0
6,Tennis Court 1,"GUEST, GUEST",75.0
7,Tennis Court 2,"GUEST, GUEST",75.0
8,Squash Court,"GUEST, GUEST",70.0
9,Massage Room 1,"Farrell, Jemima",39.6


<br>

#### Q9: This time, produce the same result 

???

<br>

#### Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [366]:
query = '''
SELECT f.name,SUM(
      CASE WHEN m.memid = 0 THEN b.slots * f.guestcost
      ELSE b.slots * f.membercost
      END) AS revenue
FROM Bookings AS b
LEFT JOIN Members AS m
ON m.memid = b.memid
LEFT JOIN Facilities AS f
ON f.facid = b.facid
GROUP BY f.name
HAVING revenue < 1000
ORDER BY revenue DESC'''

In [367]:
df = pd.read_sql_query(query, connection)
df

,name,revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


<br>

#### Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [368]:
query = '''
SELECT (m1.surname || ', ' || m1.firstname) AS recommender, (m2.surname || ', ' || m2.firstname) AS recommeded
FROM Members AS m1
INNER JOIN Members AS m2
ON m1.memid = m2.recommendedby
ORDER BY recommender'''

In [369]:
df = pd.read_sql_query(query, connection)
df

,recommender,recommeded
0,"Bader, Florence","Sarwin, Ramnaresh"
1,"Baker, Timothy","Coplin, Joan"
2,"Butters, Gerald","Genting, Matthew"
3,"Farrell, Jemima","Baker, Timothy"
4,"Farrell, Jemima","Pinker, David"
5,"Genting, Matthew","Rumney, Henrietta"
6,"Jones, David","Jones, Douglas"
7,"Joplette, Janice","Dare, Nancy"
8,"Joplette, Janice","Jones, David"
9,"Purview, Millicent","Hunt, John"


<br>

#### Q12: Find the facilities with their usage by member, but not guests

In [370]:
query = '''
SELECT f.name, ROUND(100.0 * SUM(slots)/(SELECT SUM(slots)
            FROM Bookings
            WHERE memid <> 0), 2) AS Pct_usage_by_members
FROM Bookings as b
LEFT JOIN Facilities as f
ON b.facid = f.facid
WHERE memid <> 0
GROUP BY b.facid
'''

In [371]:
df = pd.read_sql_query(query, connection)
df

,name,Pct_usage_by_members
0,Tennis Court 1,14.09
1,Tennis Court 2,12.99
2,Badminton Court,15.99
3,Table Tennis,11.69
4,Massage Room 1,13.02
5,Massage Room 2,0.80
6,Squash Court,6.16
7,Snooker Table,12.66
8,Pool Table,12.60


<br>

#### Q13: Find the facilities usage by month, but not guests

In [374]:
query = '''
SELECT 
    strftime('%m', b.starttime) AS month, 
    f.name, 
    SUM(slots) AS usage,
    ROUND(100.0 * SUM(slots)/ SUM(SUM(slots)) OVER(PARTITION BY strftime('%m', b.starttime)),2) percent_month
FROM Bookings as b 
LEFT JOIN Facilities as f
ON b.facid = f.facid
WHERE memid <> 0
GROUP BY name, month'''

In [375]:
df = pd.read_sql_query(query, connection)
df

,month,name,usage,percent_month
0,07,Badminton Court,165,15.55
1,07,Massage Room 1,166,15.65
2,07,Massage Room 2,8,0.75
3,07,Pool Table,110,10.37
4,07,Snooker Table,140,13.20
5,07,Squash Court,50,4.71
6,07,Table Tennis,98,9.24
7,07,Tennis Court 1,201,18.94
8,07,Tennis Court 2,123,11.59
9,08,Badminton Court,414,16.36
